In [9]:
import sqlite3
from datetime import datetime

# Create or connect to the database
connection = sqlite3.connect('new_libraryM.db')
c = connection.cursor()


class Admin:
    def __init__(self, username, password):#constructor
        self.username = username
        self.password = password

    def change_password(self, new_password):
        self.password = new_password

        c.execute('UPDATE admins SET password=? WHERE username=?', (new_password, self.username))
        connection.commit()

def login():
    # entering username and password from the admins
    username = input("Enter username: ")
    password = input("Enter password: ")

    c.execute('SELECT * FROM admins WHERE username=? AND password=?', (username, password))
    #fetch the next row of a query result
    admin = c.fetchone()

    if admin:
       return Admin(username, password)
    else:
        print("Invalid username or password")
        return False

def change_password_prompt(admin):
    current_password = input("Enter current password: ")
    if current_password == admin.password:
        new_password = input("Enter new password: ")
        admin.change_password(new_password)
        print("Password changed successfully")
    else:
        print("Current password is incorrect")

def add_book():
    title = input("Enter title: ")
    author = input("Enter author: ")
    isbn = input("Enter ISBN: ")
    quantity = int(input("Enter quantity: "))

    c.execute('INSERT INTO books (title, author, isbn, quantity) VALUES (?, ?, ?, ?)', (title, author, isbn, quantity))
    connection.commit()

    print("Book added successfully")

def show_books():

    c.execute('SELECT * FROM books')
    books = c.fetchall()

    if not books:
        print("No books available")
    else:
        print("\nShowing Books....")
        for book in books:
           print(f"{book[0]}. {book[1]} by {book[2]} (ISBN: {book[3]}) - Quantity: {book[4]}")

#HERE, IMPLEMENTING DATABASE WILL BE BETTER APPROACH, BUT FOR THIS PROJECT DETAILS ARE STORED IN ARRAY.

issued_books = []
def issue_books():
    def get_issued_book_count(student_rollno):
        count = 0
        for rollno, isbnC, ddt in issued_books:
            if rollno == student_rollno:
                count += 1
        return count
    isbn = input("\n\nEnter ISBN of the book to issue: ")
    student_rollno = input("Enter student roll number: ")

    if get_issued_book_count(student_rollno) >= 3:
        print("Maximum limit of issued books reached for this student.")
        return

    c.execute('SELECT * FROM books WHERE isbn=?', (isbn,))
    book = c.fetchone()

    if book:
        if book[4] > 0:#quantity greater than zero
            c.execute('UPDATE books SET quantity = quantity - 1 WHERE isbn=?', (isbn,))
            connection.commit()
            issued_books.append((student_rollno, isbn, datetime.now()))
            print("Book issued successfully")
        else:
            print("Book is out of stock")
    else:
        print("Book not found")



def edit_books():
    isbn = input("\n\nEnter ISBN of the book to edit: ")
    c.execute('SELECT * FROM books WHERE isbn=?', (isbn,))
    book = c.fetchone()

    if book:
        title = input(f"Enter new title (current: {book[1]}): ")
        author = input(f"Enter new author (current: {book[2]}): ")
        quantity = int(input(f"Enter new quantity (current: {book[4]}): "))

        c.execute('UPDATE books SET title=?, author=?, quantity=? WHERE isbn=?', (title, author, quantity, isbn))
        connection.commit()

        print("Book details updated successfully")
    else:
        print("Book not found")

def return_books():
      student_rollno = input("Enter student roll number: ")

      index = None #no any matching roll no                  #start,stop,increment
      for i in range(len(issued_books) - 1, -1, -1):#(6-1,-1,-1) from end,stop end, increment -1
          if issued_books[i][0] == student_rollno:
              index = i
              break

      if index is not None:
          roll, isbn, issue_date = issued_books.pop(index)
          return_date = datetime.now()

          days_difference = (return_date - issue_date).days
          if days_difference <= 30:
            print("Book returned successfully.")
          else:
            extra_days = days_difference - 30
            extra_charges = extra_days * 10  # Assuming 10 Rs. per day extra charges
            print(f"Book returned successfully. Extra charges of {extra_charges} Rs. applied for {extra_days} extra days.")


          c.execute('UPDATE books SET quantity = quantity + 1 WHERE isbn=?', (isbn,))
          connection.commit()

      else:
          print("No books have been issued to this student.")

def search_books():
    title = input("\n\nEnter title of the book to search: ")
    c.execute('SELECT * FROM books WHERE title LIKE ?', ('%' + title + '%',))
    books = c.fetchall()

    if books:
        for id, title, author, isbn, quantity in books:
            print(f"{title} by {author} (ISBN: {isbn}) - Quantity: {quantity}")
    else:
        print("No books found with the given title")

def delete_books():
    isbn = input("\n\nEnter ISBN of the book to delete: ")
    c.execute('SELECT * FROM books WHERE isbn=?', (isbn,))
    book = c.fetchone()

    if book:
        c.execute('DELETE FROM books WHERE isbn=?', (isbn,))
        connection.commit()
        print("Book deleted successfully")
    else:
        print("Book not found")

def main():
    admin = login()
    if not admin:
        return

    while True:
        print("\n\nDashboard:")
        now = datetime.now()

        print("Date and Time =", now)
        print("1. Add Books")
        print("2. Issue Books")
        print("3. Edit Books")
        print("4. Return Books")
        print("5. Delete Books")
        print("6. Search Books")
        print("7. Show Books")
        print("8. Change password")
        print("9. Log Out")

        choice = input("Enter your choice: ")

        if choice == "1":
            add_book()
        elif choice == "2":
            issue_books()
        elif choice == "3":
            edit_books()

        elif choice == "4":
            return_books()

        elif choice == "5":
            delete_books()

        elif choice == "6":
            search_books()

        elif choice == "7":
            show_books()
        elif choice == "8":
            change_password_prompt(admin)
        elif choice == "9":
            print("logging out......")
            break
        else:
            print("Invalid choice")

if __name__ == "__main__":
    main()

# Close the database connection
connection.close()



Enter username: admin1
Enter password: password1


Dashboard:
Date and Time = 2024-05-19 17:10:06.283083
1. Add Books
2. Issue Books
3. Edit Books
4. Return Books
5. Delete Books
6. Search Books
7. Show Books
8. Change password
9. Log Out
Enter your choice: 7

Showing Books....
2. harry potter by jk rowling (ISBN: 12) - Quantity: 2
3. kite runner by khalid (ISBN: 11) - Quantity: 4
4. War and peace by Leo (ISBN: 7) - Quantity: 5
5. Damaged by Ali (ISBN: 8) - Quantity: 6
6. The forbidden door by Dean (ISBN: 9) - Quantity: 7


Dashboard:
Date and Time = 2024-05-19 17:10:12.633877
1. Add Books
2. Issue Books
3. Edit Books
4. Return Books
5. Delete Books
6. Search Books
7. Show Books
8. Change password
9. Log Out
Enter your choice: 9
logging out......


# New Section